Step 1

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

# Create widgets for displaying images
display_color = widgets.Image(format='jpeg', width='30%')
display_depth = widgets.Image(format='jpeg', width='30%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# Convert numpy array to jpeg coded data for displaying
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

Step 2

In [ ]:
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("yolo11l_half.engine")

Step 3

In [ ]:
import traitlets
import numpy as np
import pyzed.sl as sl
import threading
import motors
from traitlets.config.configurable import SingletonConfigurable

# Initialize the Robot class
robot = motors.MotorsYukon(mecanum=False)

class Camera(SingletonConfigurable):
    color_value = traitlets.Any()  # Monitor color_value variable
    
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER

        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

        self.runtime = sl.RuntimeParameters()
        self.thread_runnning_flag = False

        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)

    def _capture_frames(self):
        while(self.thread_runnning_flag == True):
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value_BGRA = self.image.get_data()
                self.color_value = cv2.cvtColor(self.color_value_BGRA, cv2.COLOR_BGRA2BGR)
                self.depth_image = np.asanyarray(self.depth.get_data())

    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join()
            robot.stop()

camera = Camera()
camera.start()

Step 4

Stop the robot

In [ ]:
camera.stop()